<a href="https://colab.research.google.com/github/vhcarrillo/CPY16S/blob/main/CPY16S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is for the analysis of 16S amplicon sequencing form cenotes in the Yucatan Peninsula

The files are in a repo in github https://github.com/vhcarrillo/CPY16S.git

The original files listed in the repo are:

In [17]:
ls CPY16S/

CPY16S.ipynb  metadata.tsv       rep-seqs-cenotes.qza  table-freq-cenotes.qza
README.md     paper_sep18.ipynb  setup_qiime2.py       table-freq-cenotes.qzv
(qiime2-2022.2) 

: 1

*   metadata.tsv contains the metadata for the sites of study
* The raw data were procesed in by Seth in order to combine the files obteined form the sequencer because some samples were sequenced first and the others later. After combined the data, were denoised by dada2 according qiime2 documentation. The artifacts (files) obtained were:
  * rep-seqs-cenotes.qza
  * table-freq-cenotes.qza
  * table-freq-cenotes.qzv



As the artifacts have have all the samples sequenced and in three of them there are no sequences it will remove.

| Sample ID |	Sequence Count |
| --------- | -------------- |
| YB03 |	60,504 |
| YB08 |	44,902 |
| YBO4 |	43,221 |
| YB13 |	41,795 |
| MX.01 |	23,095 |
| MX.02	| 16 |
| YB05 |	12 |
| MX.04 |	0 |

In [20]:
qiime feature-table filter-samples \
  --i-table CPY16S/table-freq-cenotes.qza \
  --m-metadata-file CPY16S/metadata.tsv \
  --o-filtered-table CPY16S/sites_filtered/table-freq-cpy16s.qza

Saved FeatureTable[Frequency] to: CPY16S/sites_filtered/table-freq-cpy16s.qza
(qiime2-2022.2) 

: 1

Now, the sites with few sequences (MX.02, YB05 and MX.04) were removed. Let's create a visualization artifact to confirm that

In [22]:
qiime feature-table summarize \
  --i-table CPY16S/sites_filtered/table-freq-cpy16s.qza\
  --o-visualization CPY16S/sites_filtered//table-freq-tes-cpy16s.qzv \
  --m-sample-metadata-file CPY16S/metadata.tsv

Saved Visualization to: CPY16S/sites_filtered//table-freq-tes-cpy16s.qzv
(qiime2-2022.2) 

: 1

__Table Summary__
| Metric |	Sample |
| ------ | ------- |
| Number of samples |	5 |
| Number of features |	1,856 |
| Total frequency |	213,517 |

__Frequency per sample__
|       | Frequency |
| ------ | ----- |
| Minimum frequency | 23,095.0 |
| 1st quartile | 41,795.0 |
| Median frequency | 43,221.0 |
| 3rd quartile | 44,902.0 |
| Maximum frequency | 60,504.0 |
| Mean frequency | 42,703.4 |


Confirmed the sites removed, the sequences will remove too

In [23]:
qiime feature-table filter-seqs \
  --i-data CPY16S/rep-seqs-cenotes.qza \
  --i-table CPY16S/sites_filtered/table-freq-cpy16s.qza \
  --o-filtered-data CPY16S/sites_filtered/rep-seqs-cpy16s.qza

Saved FeatureData[Sequence] to: CPY16S/sites_filtered/rep-seqs-cpy16s.qza
(qiime2-2022.2) 

: 1

and tabulate the sequences

In [24]:
qiime feature-table tabulate-seqs \
  --i-data CPY16S/sites_filtered/rep-seqs-cpy16s.qza \
  --o-visualization CPY16S/sites_filtered/rep-seqs-cpy16s.qzv

Saved Visualization to: CPY16S/sites_filtered/rep-seqs-cpy16s.qzv
(qiime2-2022.2) 

: 1

__Sequence Length Statistics__
| Sequence Count | Min Lenght | Max Lenght | Mean Lenght | Range | Standard Deviation |
| -------------- | ---------- | ---------- | ----------- | ----- | ------------------ |
| 1856 | 189 | 273 | 252.87 | 84 | 3.29

__Phylogenetic diversity analyses__

building a phylogenetic tree

In [25]:
qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences CPY16S/sites_filtered/rep-seqs-cpy16s.qza \
    --output-dir CPY16S/tree

Saved FeatureData[AlignedSequence] to: CPY16S/tree/alignment.qza
Saved FeatureData[AlignedSequence] to: CPY16S/tree/masked_alignment.qza
Saved Phylogeny[Unrooted] to: CPY16S/tree/tree.qza
Saved Phylogeny[Rooted] to: CPY16S/tree/rooted_tree.qza
(qiime2-2022.2) 

: 1

Alpha and Beta Diversity

In [ ]:
qiime diversity core-metrics-phylogenetic \
    --i-table CPY16S/sites_filtered/table-freq-cpy16s.qza \
    --i-phylogeny CPY16S/tree/rooted_tree.qza \
    --p-sampling-depth 23094 \
    --m-metadata-file CPY16S/metadata.tsv \
    --output-dir CPY16S/diversity

/bin/bash: qiime: command not found


**Taxonomy Analisys**

We will use a Bayes classifier trained on the GreenGenes database which can be downloaded from:
https://data.qiime2.org/2021.11/common/silva-138-99-515-806-nb-classifier.qza

In [ ]:
!wget https://data.qiime2.org/2021.11/common/silva-138-99-515-806-nb-classifier.qza

--2022-04-27 17:46:30--  https://data.qiime2.org/2021.11/common/silva-138-99-515-806-nb-classifier.qza
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2021.11/common/silva-138-99-515-806-nb-classifier.qza [following]
--2022-04-27 17:46:31--  https://s3-us-west-2.amazonaws.com/qiime2-data/2021.11/common/silva-138-99-515-806-nb-classifier.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.233.96
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.233.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148294965 (141M) [binary/octet-stream]
Saving to: ‘silva-138-99-515-806-nb-classifier.qza’

silva-138-99-515-80 100%[===================>] 141.42M  41.8MB/s    in 3.4s    

2022-04-27 17:46:34 (41.8 MB/s) - ‘silva-138-9

In [ ]:
!qiime feature-classifier classify-sklearn \
    --i-reads CPY16S/sites_filtered/rep-seqs-cpy16s.qza \
    --i-classifier silva-138-99-515-806-nb-classifier.qza \
    --o-classification CPY16S/taxonomy/taxa-cpy16s.qza

/bin/bash: qiime: command not found


Now let's have a look at the relative abundances of the different bacterial taxa we have in each sample:

In [ ]:
!qiime taxa barplot \
    --i-table dada2/table.qza \
    --i-taxonomy taxa.qza \
    --m-metadata-file metadata.tsv \
    --o-visualization taxa_barplot.qzv

/bin/bash: qiime: command not found


In [ ]:
import sklearn
sklearn.__version__
sklearn.show_versions()


System:
    python: 3.7.13 (default, Apr 24 2022, 01:04:09)  [GCC 7.5.0]
executable: /usr/bin/python3
   machine: Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic

Python dependencies:
          pip: 21.1.3
   setuptools: 57.4.0
      sklearn: 1.0.2
        numpy: 1.21.6
        scipy: 1.4.1
       Cython: 0.29.28
       pandas: 1.3.5
   matplotlib: 3.2.2
       joblib: 1.1.0
threadpoolctl: 3.1.0

Built with OpenMP: True


The scikit-learn version (0.24.1) used to generate this artifact does not match the current version of scikit-learn installed (0.22.1).